In [52]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [21]:
URL = 'https://www.amazon.com/s?k=playstation+5&crid=3BV45YDFQQE2G&sprefix=playstation+4%2Caps%2C269&ref=nb_sb_noss_2'

In [22]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36', 'Accept-Language':'en-US,en;q=0.5'})

In [23]:
webpage = requests.get(URL, headers=HEADERS)

In [24]:
type(webpage.content)

bytes

In [25]:
soup = BeautifulSoup(webpage.content, 'html.parser')

In [26]:
links = soup.find_all('a', attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

In [48]:
links[0]

<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/PlayStation%C2%AE5-console-slim-PlayStation-5/dp/B0CL61F39H/ref=sr_1_1?crid=3BV45YDFQQE2G&amp;dib=eyJ2IjoiMSJ9.IBzp2a7hJYOBBKGDpX2BCOMq4I8axp2PbKuGczvmrI4LUcaBhliHEKJct11Q19JAQWlsHxm1MKPIVXWfwgMhHhtP4GL7I8MgFcL6_UOdY-Jg_qGT1LInF9pbCtwsqEWqI65TAtP65C5I8yweIsazejyvx-RWl5UNG5KkN1c57JPcCwiq7cAVEuDx7D9IFuZnzBSjPisqJD95cNFD8XJfFhI2XdDwbywYDxoYupiZq7g.2daAn_91revvBQErNmtzIVY_aWIn6FuluuOBQbRzrGU&amp;dib_tag=se&amp;keywords=playstation+5&amp;qid=1715434661&amp;sprefix=playstation+4%2Caps%2C269&amp;sr=8-1"><span class="a-size-medium a-color-base a-text-normal">PlayStation®5 console (slim)</span> </a>

In [31]:
link = links[0].get('href')

In [32]:
product_link = 'https://www.amazon.com' + link

In [33]:
product_link

'https://www.amazon.com/PlayStation%C2%AE5-console-slim-PlayStation-5/dp/B0CL61F39H/ref=sr_1_1?crid=3BV45YDFQQE2G&dib=eyJ2IjoiMSJ9.IBzp2a7hJYOBBKGDpX2BCOMq4I8axp2PbKuGczvmrI4LUcaBhliHEKJct11Q19JAQWlsHxm1MKPIVXWfwgMhHhtP4GL7I8MgFcL6_UOdY-Jg_qGT1LInF9pbCtwsqEWqI65TAtP65C5I8yweIsazejyvx-RWl5UNG5KkN1c57JPcCwiq7cAVEuDx7D9IFuZnzBSjPisqJD95cNFD8XJfFhI2XdDwbywYDxoYupiZq7g.2daAn_91revvBQErNmtzIVY_aWIn6FuluuOBQbRzrGU&dib_tag=se&keywords=playstation+5&qid=1715434661&sprefix=playstation+4%2Caps%2C269&sr=8-1'

In [34]:
new_webpage = requests.get(product_link, headers=HEADERS)

In [35]:
new_soup = BeautifulSoup(new_webpage.content, 'html.parser')

In [39]:
new_soup.find('span', attrs={'id':'productTitle'}).text.strip()

'PlayStation®5 console (slim)'

In [46]:
new_soup.find('span', attrs={'class':'a-price a-text-price a-size-medium apexPriceToPay'}).find("span",attrs={'a-offscreen'}).text

'$479.04'

In [47]:
new_soup.find("span", attrs={"class":'a-icon-alt'}).text

'4.7 out of 5 stars'

## Iterating through every product to create the dataframe

In [55]:
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [56]:
# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

In [57]:
# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

In [58]:
#Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

In [59]:
# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [65]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [66]:
amazon_df

,title,price,rating,reviews,availability
1,Sony Playstation PS4 1TB Black Console,,4.6 out of 5 stars,"1,462 ratings",In Stock
2,PlayStation®4 Console – Call of Duty® Modern W...,,4.5 out of 5 stars,62 ratings,Only 4 left in stock - order soon.
3,"Playstation SONY 4, 500GB Slim System [CUH-221...",,4.5 out of 5 stars,419 ratings,Only 2 left in stock - order soon.
4,Flagship Newest Play Station 4 1TB HDD Only on...,,4.5 out of 5 stars,204 ratings,Not Available
5,PlayStation 4 500GB Console [Old Model][Discon...,,4.6 out of 5 stars,"13,579 ratings",Only 1 left in stock - order soon.
6,Sony PlayStation 4 Pro 1TB Console - Black (PS...,,4.5 out of 5 stars,"4,051 ratings",Only 1 left in stock - order soon.
7,"Playstation Sony 4, 500GB Slim System [CUH-221...",,4.5 out of 5 stars,343 ratings,Only 1 left in stock - order soon.
8,PlayStation®5 Digital Edition (slim),,4.7 out of 5 stars,"5,027 ratings",Only 8 left in stock - order soon.
9,PlayStation 4 Slim 500GB Console [Discontinued],,4.5 out of 5 stars,"1,893 ratings",In Stock
10,PlayStation 4 Console 500 GB (Renewed),,5.0 out of 5 stars,3 ratings,Not Available
